# TMA4220 Project 2
Group 10: Nikolai Rasmus Sætren, August Hardersen Skogøy

In [1]:
import ngsolve
from netgen.geom2d import unit_square
import netgen.gui
from netgen.meshing import *
from ngsolve.webgui import Draw
import numpy as np
from ngsolve import dx, grad

%gui tk

$$
\begin{align*}
(\partial_t \phi, v) + (M \nabla \mu, \nabla v) &= 0 \quad \text{for all } v \in V, \\
(\mu, w) - \left( \gamma (\phi^3 - \phi), w \right) - \left( \epsilon \nabla \phi, \nabla w \right) &= 0 \quad \text{for all } w \in V.
\end{align*}

$$

In [2]:
# Defining constants

epsilon = 0.02
gamma = 50
M = 1

h_max = 0.01

In [3]:
# Setting up the mesh

ngmesh = Mesh(dim=2)

N= int(1/h_max)

"""
Code for the square mesh taken from the documentation page
https://docu.ngsolve.org/latest/i-tutorials/unit-4.3-manualmesh/manualmeshing.html
"""


pnums = []
for i in range(N + 1):
    for j in range(N + 1):
        pnums.append(ngmesh.Add(MeshPoint(Pnt(i / N, j / N, 0))))

idx_dom = ngmesh.AddRegion("mat", dim=2)
for j in range(N):
    for i in range(N):
        ngmesh.Add(Element2D(idx_dom, [pnums[i + j * (N + 1)],
                               pnums[i + (j + 1) * (N + 1)],
                               pnums[i + 1 + (j + 1) * (N + 1)],
                               pnums[i + 1 + j * (N + 1)]]))
        
# horizontal boundaries
for i in range(N):
   ngmesh.Add(Element1D([pnums[N + i * (N + 1)],
                       pnums[N + (i + 1) * (N + 1)]], index=1))
   ngmesh.Add(Element1D([pnums[0 + i * (N + 1)], pnums[0 + (i + 1) * (N + 1)]], index=1))

# vertical boundaries
for i in range(N):
   ngmesh.Add(Element1D([pnums[i], pnums[i + 1]], index=2))
   ngmesh.Add(Element1D([pnums[i + N * (N + 1)], pnums[i + 1 + N * (N + 1)]], index=2))

mesh = ngsolve.Mesh(ngmesh)

Fully explicit:
$$
\begin{align*}
\left( \frac{\phi^{n+1} - \phi^n}{\tau}, v \right) + (M \nabla \mu^{n+1}, \nabla v) &= 0 \quad \text{for all } v \in V, \\
(\mu^{n+1}, w) - (\epsilon \nabla \phi^n, \nabla w) &= \left( \gamma ((\phi^n)^3 - \phi^n), w \right) \quad \text{for all } w \in V.
\end{align*}

$$

which can be reformulated as

$$
\begin{align*}
\left( \frac{\phi^{n+1} }{\tau}, v \right) &=  \left( \frac{\phi^n}{\tau}, v \right) - (M \nabla \mu^{n+1}, \nabla v)  \quad \text{for all } v \in V, \\
(\mu^{n+1}, w)  &= (\epsilon \nabla \phi^n, \nabla w) + \left( \gamma ((\phi^n)^3 - \phi^n), w \right) \quad \text{for all } w \in V.
\end{align*}
$$

In [4]:
# Fully Explicit Scheme

h_max = 0.05
tau = 1e-5


# Defining finite element space

V_phi = ngsolve.H1(mesh,order=1)
V_mu  = ngsolve.H1(mesh,order=1)

phi, v = V_phi.TnT()
mu, w = V_mu.TnT()

gfu = ngsolve.GridFunction(V_phi)
gfu_mu = ngsolve.GridFunction(V_mu)


# Initialisation of phi_0

phi_0 = np.random.uniform(low = -1, high = 1, size = mesh.nv)
rand_expr = ngsolve.GridFunction(V_phi)
for i in range(mesh.nv):
    rand_expr.vec[i] = phi_0[i]

gfu.Set(rand_expr) 

# Bilinear forms:

a_phi = ngsolve.BilinearForm(V_phi)
a_phi += (1/tau)*phi*v*dx

a_mu = ngsolve.BilinearForm(V_mu)
a_mu += mu*w*dx

a_phi.Assemble()
a_mu.Assemble()

# Since it's fully explicit we get linear forms on the rhs (mu is updated before phi)
# rhs also depends on previous iteration and as such these linear forms must be updated every iteration

def rhs_mu(gfu):
    f_mu = ngsolve.LinearForm(V_mu)
    f_mu += (epsilon*grad(gfu)*grad(w) + gamma*((gfu**3) - gfu)*w)*dx # Minus epsilon?
    f_mu.Assemble()
    return f_mu


def rhs_phi(gfu_old, gfu_mu):
    f_phi = ngsolve.LinearForm(V_phi)
    f_phi += (1/tau)*gfu_old*v*dx - M*grad(gfu_mu)*grad(v)*dx
    f_phi.Assemble()
    return f_phi





In [ ]:
iter = 10


Draw(gfu)
for n in range(iter):

    # Solving for mu
    f_mu = rhs_mu(gfu)
    gfu_mu.vec.data = a_mu.mat.Inverse(V_mu.FreeDofs())*f_mu.vec

    # Solving for phi
    f_phi = rhs_phi(gfu, gfu_mu)
    gfu.vec.data = a_phi.mat.Inverse(V_phi.FreeDofs())*f_phi.vec

    

Implicit-explicit (IMEX)

$$
\begin{align*}
\left( \frac{\phi^{n+1} - \phi^n}{\tau}, v \right) + (M \nabla \mu^{n+1}, \nabla v) &= 0 \quad \text{for all } v \in V, \\
(\mu^{n+1}, w) - \left( \gamma \left[(\phi^n)^2 - 1\right] \phi^{n+1}, w \right) - (\epsilon \nabla \phi^{n+1}, \nabla w) &= 0 \quad \text{for all } w \in V.
\end{align*}


$$


In [9]:
# IMEX

epsilon = 0.02
gamma = 50
M = 1

h_max = 0.05
tau = 1e-5


# Defining finite element space

V_phi = ngsolve.H1(mesh,order=1)
V_mu  = ngsolve.H1(mesh,order=1)

V = ngsolve.FESpace([V_phi, V_mu])  # Mixed finite element space
(u_phi, u_mu), (v_phi, v_mu) = V.TnT()  # Trial and test functions

gfu_phi = ngsolve.GridFunction(V_phi)
gfu_mu = ngsolve.GridFunction(V_mu)


# Initialisation of phi_0

phi_0 = np.random.uniform(low = -1, high = 1, size = mesh.nv)
rand_expr = ngsolve.GridFunction(V_phi)
for i in range(mesh.nv):
    rand_expr.vec[i] = phi_0[i]

gfu_phi.Set(rand_expr) 

# Bilinear form

def assemble_A(gfu_phi):
    A = ngsolve.BilinearForm(V)
    A += (1 / tau) * u_phi * v_phi * dx  
    A += M * grad(u_mu) * grad(v_phi) * dx  
    A += u_mu * v_mu * dx  
    A += -epsilon * grad(u_phi) * grad(v_mu) * dx  
    A += -gamma * ((gfu_phi**2 - 1) * u_phi * v_mu) * dx  
    A.Assemble()
    return A

# linear form

def assemble_F(gfu_phi):
    F = ngsolve.LinearForm(V)
    F += (1 / tau) * gfu_phi * v_phi * dx  
    F.Assemble()
    return F

gfu = ngsolve.GridFunction(V)  # Mixed solution vector [phi, mu]


In [ ]:
# Time-stepping loop
iter = 100
for n in range(iter):
    # Update RHS
    F = assemble_F(gfu_phi)
    A = assemble_A(gfu_phi)
    
    # Solve the coupled system
    gfu.vec.data = A.mat.Inverse(V.FreeDofs()) * F.vec

    # Extract phi and mu components
    gfu_phi.vec.data = gfu.components[0].vec
    gfu_mu.vec.data = gfu.components[1].vec


Draw(gfu_phi)

Fully Implicit

$$
\begin{align*}
\left( \frac{\phi^{n+1} - \phi^n}{\tau}, v \right) + (M \nabla \mu^{n+1}, \nabla v) &= 0 \quad \text{for all } v \in V, \\
(\mu^{n+1}, w) - \left( \gamma ((\phi^{n+1})^3 - \phi^{n+1}), w \right) - (\epsilon \nabla \phi^{n+1}, \nabla w) &= 0 \quad \text{for all } w \in V.
\end{align*}

$$
